<a href="https://colab.research.google.com/github/jeffheaton/app_deep_learning/blob/main/assignments/assignment_yourname_class3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/github/grfiv/pytorch_class/assignments/assignment_GeorgeFisher_EarlyStopping_class3.ipynb

# T81-558: Applications of Deep Neural Networks with early stopping
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/index.html)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 3 Assignment: Simple Classification Neural Network** WITH EARLY STOPPING

**Student Name: George Fisher**

# Assignment Instructions

For this assignment, you will use the **crx** dataset. You can find the CSV file on my data site, at this location: [crx](https://data.heatonresearch.com/data/t81-558/crx.csv). Load and summarize the data set.  You will submit this summarized dataset to the **submit** function.  See [Assignment #1](https://github.com/jeffheaton/app_deep_learning/blob/main/assignments/assignment_yourname_class1.ipynb) for details on how to submit an assignment or check that one was submitted.

The RAW datafile looks something like the following:

|a1|a2|s3|a4|a5|a6|a7|a8|a9|a10|a11|a12|a13|a14|a15|a16|
|--|--|--|--|--|--|--|--|--|---|---|---|---|---|---|---|
|b|30.83|0|u|g|w|v|1.25|t|t|1|f|g|202|0|+|
|a|58.67|4.46|u|g|q|h|3.04|t|t|6|f|g|43|560|+|
|...|...|...|...|...|...|...|...|...|...|...|...|...|...|...|...|

For this assignment you must complete the following.

* Write a classification neural network that predicts the probability of either "+" or "-" for the column **a16**.
* Use early stopping to know when to complete your training.
* For all columns that are categorical, you must convert them to dummy variables.
* Some columns have missing values, fill these missing values with the median of that column.
* This is a simple neural network using basic techniques, do not worry too much about overall accuracy.
* Predict/submit for the entire dataset that I gave you, training and validation, you should have the same number of rows as crx.csv (690 data and 1 header row).

Your submit will look something like the following:

|+|-|
|-|-|
|0.09405358|0.90594643|
|0.33253232|0.66746765|
|0.098494485|0.90150553|
|...|...|

Common errors that you may run into include:

* **ValueError: could not convert string to float: ...** - Value errors typically mean you've not converted all of the categoricals to dummy variables.

* **tloss nan:** - Nan's usually mean youve not filled all missing values.



# Google CoLab Instructions

If you are using Google CoLab, it will be necessary to mount your GDrive so that you can send your notebook during the submit process. Running the following code will map your GDrive to ```/content/drive```.

In [5]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Early stopping (see module 3.4)
import copy

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
import torch

device = (
    "mps"
    if torch.backends.mps.is_built()
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Note: not using Google CoLab
Using device: mps


# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems.

**It is unlikely that should need to modify this function.**

# Assignment #3 Sample Code

The following code provides a starting point for this assignment.

In [7]:
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import tqdm

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/crx.csv",na_values=['?'])

le  = preprocessing.LabelEncoder()
y   = le.fit_transform(df["a16"])
a16 = le.classes_
print("a16:", a16)

for col in ["a1","a4","a5","a6","a7","a9","a10","a12","a13"]:
    dummies  = pd.get_dummies(df[col], prefix=col + "_").astype(int)
    df       = pd.concat([df, dummies], axis=1)
    df.drop(col, axis=1, inplace=True)
    

for col in ["a2","s3","a8","a11","a14","a15"]:
    med     = df[col].median()
    df[col] = df[col].fillna(med)
    
x_columns = df.columns.drop("a16")
x         = df[x_columns].values

# Split into validation and training sets
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42)

# Numpy to Torch Tensor
x_train = torch.tensor(x_train, device=device, dtype=torch.float32)
y_train = torch.tensor(y_train, device=device, dtype=torch.long)

x_test = torch.tensor(x_test, device=device, dtype=torch.float32)
y_test = torch.tensor(y_test, device=device, dtype=torch.long)

a16: ['+' '-']


In [9]:
# Create datasets
BATCH_SIZE = 16

dataset_train    = TensorDataset(x_train, y_train)
dataloader_train = DataLoader(
    dataset_train, batch_size=BATCH_SIZE, shuffle=True)

dataset_test    = TensorDataset(x_test, y_test)
dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=True)

In [11]:
# Create model using nn.Sequential
model = nn.Sequential(
    nn.Linear(x_train.shape[1], 50),
    nn.ReLU(),
    nn.Linear(50, 25),
    nn.ReLU(),
    nn.Linear(25, len(a16)),
    #nn.LogSoftmax(dim=1), # Implied by use of CrossEntropyLoss
)

model = torch.compile(model,backend="aot_eager").to(device)

loss_fn = nn.CrossEntropyLoss()  # cross entropy loss

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
es = EarlyStopping()


# model.train()
# for epoch in range(3000):
#     optimizer.zero_grad()
#     out = model(x)
#     # Note: CrossEntropyLoss combines nn.LogSoftmax() and nn.NLLLoss() so don't use Softmax in the model
#     loss = criterion(out, y)
#     loss.backward()
#     optimizer.step()

#     if epoch % 100 == 0:
#         print(f"Epoch {epoch}, loss: {loss.item()}")
epoch = 0
done  = False
while epoch < 2000 and not done:
    epoch += 1
    steps = list(enumerate(dataloader_train))
    pbar = tqdm.tqdm(steps)
    model.train()
    for i, (x_batch, y_batch) in pbar:
        y_batch_pred = model(x_batch.to(device))
        loss = loss_fn(y_batch_pred, y_batch.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss, current = loss.item(), (i + 1) * len(x_batch)
        if i == len(steps) - 1:
            model.eval()
            pred = model(x_test)
            vloss = loss_fn(pred, y_test)
            if es(model, vloss):
                done = True
            pbar.set_description(
                f"Epoch: {epoch}, tloss: {loss}, vloss: {vloss:>7f}, {es.status}"
            )
        else:
            pbar.set_description(f"Epoch: {epoch}, tloss {loss:}")

RuntimeError: Dynamo is not supported on Python 3.12+

In [55]:
model.eval()
pred               = model(x)
_, predict_classes = torch.max(pred, 1)

In [67]:
correct = accuracy_score(y.cpu().detach(), predict_classes.cpu().detach())
print(f"Accuracy: {correct}")

Accuracy: 0.9884057971014493


In [69]:
print(f"Predictions: {predict_classes}")
print(f"Expected: {y}")

Predictions: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1